# 动手学深度学习

- Forked by [李沐-动手学深度学习](https://zh.d2l.ai/chapter_deep-learning-computation/model-construction.html)

In [1]:
import torch
import torch.nn as nn


## 5. 深度学习计算

- 5.1 层和块
- 5.2 参数管理
- 5.3 延后初始化
- 5.4 自定义层
- 5.5 读写文件
- 5.6 GPU

### 5.5 读写文件

1. 保存网络结构和参数

- 当网络结构发生变化时，torch.save()保存的模型，在load的时候相比于state_dict会遇到一些麻烦
- 采用sequential定义的网络，可以采用net[0]这种方式访问和保存单层的参数


In [ ]:
model = nn.Sequential(nn.Linear(10,100), nn.ReLU(), nn.Linear(100,10))
torch.save(model, 'model.pt')
m = torch.load('model.pt')

2. state_dict()返回的是有序字典，可以通过字典的键值访问想要的参数。

In [ ]:
torch.save(net.hidden.state_dict(), 'mlp.hidden.params')
clone = MLP()
clone.hidden.load_state_dict(torch.load('mlp.hidden.params'))

In [ ]:
clone_hidden.load_state_dict(torch.load('hidden.params'),strict=False)

3. 从train模式调整为test模型，不再进行训练更改梯度

4. 假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如说，如果你想在一个新的网络中使用之前网络的前两层，你该怎么做？

In [ ]:
# 如何保存和加载模型的某层参数
torch.save(net.hidden.state_dict(), 'mlp.hidden.params')
clone = MLP()
clone.hidden.load_state_dict(torch.load('mlp.hidden.params'))
print(clone.hidden.weight == net.hidden.weight)

5. 如果你是用类实现的网络结构，那么你在别的文件里load的时候需要重新实现一下类，否则load不了，找不到那个类

### 5.6 GPU

1. 应该如何在GPU上读写模型参数

In [ ]:
net.to(device=torch.device('cuda'))

## 8. 循环神经网络

- 8.1 序列模型
- 8.2 文本预处理
- 8.3 语言模型和数据集
- 8.4 循环神经网络
- 8.5 循环神经网络从零实现
- 8.6 循环神经网络的简洁实现
- 8.7 通过时间反向传播

## 10. 注意力机制

- 10.1 注意力提示
- 10.2 注意力汇聚
- 10.3 注意力评分函数
- 10.4 Bahdanau注意力
- 10.5 多头注意力
- 10.6 自注意力和位置编码
- 10.7 Transformer

### 10.1 注意力提示

1. two-component框架：基于非自主性和自主性提示，有选择地引导注意力的焦点
    - 非自主性提示：基于环境中物体的突出性和易见性
    - 自主性提示：由于突出性导致的选择不同，而后选择的物体受到了认知和意识的控制


2. 查询、键、值
    - “是否包含自主性提示”将注意力机制与全连接层或汇聚层区别开来
    - 自主性提示被称为查询 -> query
    - 给定任何查询，注意力机制通过注意力汇聚(attention pooling)，将选择引导至感官输入(sensory inputs, 例如中间特征表示) -> 值, value
    - 每一个值都与一个键 -> key 配对，可以作为感官输入的非自主提示，引导得出最匹配的值
    
    ![注意力机制](./asserts/注意力QKV.PNG)

    - 想去寻找物体中最特别的一个东西(自主提示 -> 任务)，红苹果在黑白球中具有最吸引人的特征(非自主提示) -> 找到红苹果(输入, 值)


3. 注意力的可视化

    - 注意力汇聚得到的是加权平均的总和，其中权重是在给定的查询和不同的键之间计算得到的